# Modelagem

In [0]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np
from nltk.corpus import stopwords
from sklearn.tree import DecisionTreeClassifier as DT

In [2]:
df = pd.read_csv('https://s3.amazonaws.com/automl-example/produtos.csv', delimiter=';', encoding='utf8')

df.dropna(inplace=True)

df.head(5)

,nome,descricao,categoria
0,O Hobbit - 7ª Ed. 2013,Produto NovoBilbo Bolseiro é um hobbit que lev...,livro
1,Livro - It A Coisa - Stephen King,Produto NovoDurante as férias escolares de 195...,livro
2,Box As Crônicas De Gelo E Fogo Pocket 5 Li...,Produto NovoTodo o reino de Westeros ao alcanc...,livro
3,Box Harry Potter,Produto Novo e Físico A série Harry Potter ch...,livro
4,Livro Origem - Dan Brown,Produto NovoDe Onde Viemos? Para Onde Vamos? R...,livro


In [3]:
df.categoria.value_counts()

livro        838
maquiagem    788
brinquedo    668
game         622
Name: categoria, dtype: int64

In [0]:
df['full_text'] = df['nome'] + ' ' + df['descricao']

In [0]:
#nltk.download('stopwords')
stopWords = set(stopwords.words('portuguese'))wtg

# CountVectorizer

In [33]:
vetor_count = CountVectorizer(lowercase=True, ngram_range=(2,2), stop_words=stopWords)
vetor_count.fit(df.full_text)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(2, 2), preprocessor=None,
        stop_words={'estávamos', 'do', 'sua', 'foi', 'depois', 'nem', 'teu', 'estiver', 'for', 'forem', 'esta', 'haja', 'aquele', 'num', 'estiverem', 'houvéssemos', 'isso', 'por', 'nas', 'hão', 'dos', 'meu', 'havemos', 'você', 'nossas', 'estamos', 'para', 'seu', 'aos', 'estejam', 'há', 'era', 'sem', 'estava..., 'hajamos', 'seja', 'seria', 'será', 'pelos', 'elas', 'me', 'tivessem', 'tiver', 'tivemos', 'dele'},
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [0]:
vetor_count_palavras = vetor_count.transform(df.full_text)

In [35]:
vetor_count_palavras.get_shape()

(2916, 145409)

In [36]:
modelo = DT()

modelo.fit(vetor_count_palavras, df['categoria'])

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [37]:
teste = vetor_count.transform(['História que conta sobre um amor impossível.'])

modelo.predict(teste)

array(['livro'], dtype=object)

# TF-IDF

In [38]:
vetor_tfidf = TfidfVectorizer(lowercase=True, ngram_range=(1,1), stop_words=stopWords)
vetor_tfidf.fit(df.full_text)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words={'estávamos', 'do', 'sua', 'foi', 'depois', 'nem', 'teu', 'estiver', 'for', 'forem', 'esta', 'haja', 'aquele', 'num', 'estiverem', 'houvéssemos', 'isso', 'por', 'nas', 'hão', 'dos', 'meu', 'havemos', 'você', 'nossas', 'estamos', 'para', 'seu', 'aos', 'estejam', 'há', 'era', 'sem', 'estava..., 'hajamos', 'seja', 'seria', 'será', 'pelos', 'elas', 'me', 'tivessem', 'tiver', 'tivemos', 'dele'},
        strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [0]:
vetor_tfidf_palavras = vetor_tfidf.transform(df.full_text)

In [40]:
modelo_2 = DT()

modelo_2.fit(vetor_tfidf_palavras, df['categoria'])

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [41]:
teste = vetor_tfidf.transform(['História que conta sobre um amor impossível.'])

modelo_2.predict(teste)

array(['livro'], dtype=object)

In [42]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(vetor_tfidf_palavras, df['categoria'])

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [43]:
lr.classes_

array(['brinquedo', 'game', 'livro', 'maquiagem'], dtype=object)

In [32]:
teste

<1x3674 sparse matrix of type '<class 'numpy.float64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [44]:
lr.predict_proba(teste)

array([[0.0868431 , 0.08122301, 0.74606555, 0.08586834]])

In [45]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()

nb.fit(vetor_tfidf_palavras, df['categoria'])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [46]:
nb.classes_

array(['brinquedo', 'game', 'livro', 'maquiagem'], dtype='<U9')

In [47]:
nb.predict_proba(teste)

array([[0.03225515, 0.08248816, 0.83535631, 0.04990038]])